# SQL Log NASA

#### Mª del Val Huerta Pintado

In [1]:
from pyspark import SparkContext, Row

In [2]:
sc = SparkContext("local", "First App")

In [3]:
data_file = "./apache.access.log_small"
raw_data = sc.textFile(data_file)

In [4]:
raw_data.count()

3432

In [5]:
raw_data.take(5)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

### 1.- Parsear el dataset para su tratamiento:

Se emplea la expresión regular

In [6]:
import re

In [7]:
def parse_log2(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)


In [8]:
def map_log(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    return(match.groups())
parsed_rdd = raw_data.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0])
parsed_def = parsed_rdd.map(lambda line: map_log(line))

In [9]:
raw_data.map(lambda line: parse_log2(line)).take(2)

[('in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
  1),
 ('uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
  1)]

* Las expresiones regulares se inician con comillas simples: '
* ^ indica que la estructura tiene que cumplirse desde el inicio
* \S simboliza todo lo que no sea espacio. El + indica que tiene que aparecer al menos uno de los elementos que estoy diciendo
* Los corchetes indica ue de aparecer algo, apareceria alguno de los elementos dentro del corchete
* \d{4} significa que aparece exactamente cuatro dígitos
* Los paréntesis para lo único que sirven es para definir los bloques que luego utiliza el metodo groups
* s minuscula es espacio
* D mayuscula es todo lo que no es digito
* (*) puede haber el caracter que has dicho y puede aparecer varias veces
* w son letras
* \w{3} coge tres numeros

In [10]:
parsed_def.take(1)

[('in24.inetnebr.com',
  '-',
  '-',
  '01/Aug/1995:00:00:01',
  '0400',
  'GET',
  '/shuttle/missions/sts-68/news/sts-68-mcc-05.txt',
  'HTTP/1.0"',
  None,
  '200',
  '1839')]

### 2.- Transformar en sqldataframe:

In [11]:
from pyspark.sql import SQLContext
from pyspark.sql import Row # Row sirve para darle nombres a cada elemento de la lista
from pyspark.sql.functions import * 

In [12]:
sqlContext = SQLContext(sc)

In [13]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x) 
    if x =="":
        return 0
    else:
        return int(x)
parsed_def.map(lambda line: convert_long(line[-1])).stats()
# stats es un metodo que nos da un summary de los datos

(count: 3432, mean: 16051.863636363621, stdev: 53247.8157482, max: 887988.0, min: 0.0)

In [14]:
sql_data = parsed_def.map(lambda p: Row(Hosts = p[0],
                                        date = p[3][:2],
                                        Endpoint = p[6], 
                                        Code = p[-2], 
                                        size = p[-1]))

In [15]:
sql_data.take(1)

[Row(Code='200', Endpoint='/shuttle/missions/sts-68/news/sts-68-mcc-05.txt', Hosts='in24.inetnebr.com', date='01', size='1839')]

In [16]:
lognasa_df = sqlContext.createDataFrame

In [17]:
df = sqlContext.createDataFrame(sql_data)
df.registerTempTable("datos")

In [18]:
result = sqlContext.sql("""
    SELECT * 
    FROM datos 
    LIMIT 10
""")
result.show()

+----+--------------------+--------------------+----+-----+
|Code|            Endpoint|               Hosts|date| size|
+----+--------------------+--------------------+----+-----+
| 200|/shuttle/missions...|   in24.inetnebr.com|  01| 1839|
| 304|                   /|     uplherc.upl.com|  01|    0|
| 304|/images/ksclogo-m...|     uplherc.upl.com|  01|    0|
| 304|/images/MOSAIC-lo...|     uplherc.upl.com|  01|    0|
| 304|/images/USA-logos...|     uplherc.upl.com|  01|    0|
| 200|/images/launch-lo...|ix-esc-ca2-07.ix....|  01| 1713|
| 304|/images/WORLD-log...|     uplherc.upl.com|  01|    0|
| 200|/history/skylab/s...|slppp6.intermind.net|  01| 1687|
| 200|/images/launchmed...|piweba4y.prodigy.com|  01|11853|
| 200|/history/skylab/s...|slppp6.intermind.net|  01| 9202|
+----+--------------------+--------------------+----+-----+



### 3.- Responder a las preguntas: 

__3.1 Mínimo, máximo y media del tamaño de las peticiones (size)__

In [19]:
Ejercicio1 = sqlContext.sql("""
    SELECT MIN(size) AS Minimo, MAX(size) AS Maximo, AVG (size) AS Media 
    FROM datos """)
Ejercicio1.show()

+------+------+------------------+
|Minimo|Maximo|             Media|
+------+------+------------------+
|     -| 99942|16183.900117508812|
+------+------+------------------+



__3.2 Nº de peticiones de cada código de respuesta (response_code)__

In [20]:
Ejercicio2 = sqlContext.sql("""
    SELECT COUNT (size) as peticiones, code AS codigo
    FROM datos
    GROUP BY code
""")

Ejercicio2.show()

+----------+------+
|peticiones|codigo|
+----------+------+
|      3140|   200|
|        50|   302|
|        22|   404|
|         1|   403|
|       219|   304|
+----------+------+



__3.3 Mostrar 20 hosts que han sido visitados más de 10 veces__

In [21]:
Ejercicio3 = sqlContext.sql ("""
    SELECT Hosts, size as visitas
    FROM datos 
    WHERE size >10 
    ORDER BY Hosts
    LIMIT 10

""")
Ejercicio3.show()


+-------------+-------+
|        Hosts|visitas|
+-------------+-------+
|128.135.36.35|  11853|
|128.135.36.35|   8677|
|128.135.36.35|    786|
|128.135.36.35|   1204|
|128.135.36.35|   7280|
|128.135.36.35|   5866|
|128.135.36.35|    786|
|128.135.36.35|    363|
|128.135.36.35|    234|
|128.135.36.35|    669|
+-------------+-------+



__3.4 Mostrar los 10 endpoints más visitados__

In [22]:
Ejercicio4 = sqlContext.sql ( """
    SELECT Endpoint, (size) AS visitas
    FROM datos
    ORDER BY (size)DESC
    LIMIT 10
""")

Ejercicio4.show()


+--------------------+-------+
|            Endpoint|visitas|
+--------------------+-------+
|/history/apollo/a...|  99942|
|/history/apollo/a...|  99942|
|/shuttle/missions...|   9978|
|/shuttle/missions...|   9978|
|/shuttle/missions...|   9978|
|   /elv/elvhead3.gif|   9925|
|   /elv/elvhead3.gif|   9925|
|   /elv/elvhead3.gif|   9925|
|/software/winvn/w...|   9866|
|/software/winvn/w...|   9866|
+--------------------+-------+



__3.5 Mostrar los 10 endpoints más visitados que no tienen código de respuesta 200__

In [23]:
Ejercicio5 = sqlContext.sql ("""
    SELECT Endpoint, size AS visitas
    FROM datos
    WHERE code != 200
    ORDER BY size DESC
    LIMIT 10
""")

Ejercicio5.show()

+--------------------+-------+
|            Endpoint|visitas|
+--------------------+-------+
|/cgi-bin/imagemap...|     98|
|/cgi-bin/imagemap...|     98|
|/cgi-bin/imagemap...|     98|
|/cgi-bin/imagemap...|     98|
|/cgi-bin/imagemap...|     97|
|/cgi-bin/imagemap...|     97|
|/cgi-bin/imagemap...|     96|
|/cgi-bin/imagemap...|     96|
|/cgi-bin/imagemap...|     96|
|/cgi-bin/imagemap...|     96|
+--------------------+-------+



__3.6 Calcular el nº de hosts distintos__

In [24]:
Ejercicio6 = sqlContext.sql ("""
    SELECT COUNT (Hosts), Hosts
    FROM datos
    GROUP BY (Hosts)
    ORDER BY (Hosts) DESC

""")

Ejercicio6.show()

+------------+--------------------+
|count(Hosts)|               Hosts|
+------------+--------------------+
|           3|ykline6.internort...|
|          10|xslip47.csrv.uida...|
|          20|wyndmoor1-7.slip....|
|          13|    wwwproxy.info.au|
|           6|www-d4.proxy.aol.com|
|          27|www-d3.proxy.aol.com|
|          13|www-d2.proxy.aol.com|
|          39|www-d1.proxy.aol.com|
|           2|www-c8.proxy.aol.com|
|          11|www-c6.proxy.aol.com|
|           6|www-c4.proxy.aol.com|
|          17|www-c3.proxy.aol.com|
|           1|www-c2.proxy.aol.com|
|          30|www-c1.proxy.aol.com|
|          28|www-b5.proxy.aol.com|
|          27|www-b3.proxy.aol.com|
|          40|www-b2.proxy.aol.com|
|          10|www-a2.proxy.aol.com|
|          23|www-a1.proxy.aol.com|
|           9|wtaskr1.ccinet.ab.ca|
+------------+--------------------+
only showing top 20 rows



__3.7 Calcular el nº de hosts cada día__

In [25]:
Ejercicio7 = sqlContext.sql (""" 
    SELECT COUNT (Hosts), date AS dia
    FROM datos
    GROUP BY date 
""")

Ejercicio7.show()

+------------+---+
|count(Hosts)|dia|
+------------+---+
|        3432| 01|
+------------+---+



__3.8 Calcular la media de peticiones diarias por host__

In [26]:
Ejercicio8 = sqlContext.sql("""
    SELECT AVG (size) AS Media_de_peticiones, date AS Dia, Hosts
    FROM datos
    GROUP BY date, Hosts

""")

Ejercicio8.show()

+-------------------+---+--------------------+
|Media_de_peticiones|Dia|               Hosts|
+-------------------+---+--------------------+
|            39762.6| 01|sunset.somcau.siu...|
|            15974.0| 01|    igate.uswest.com|
|             2191.0| 01|kgtyk4.kj.yamagat...|
|  21095.53846153846| 01|    wwwproxy.info.au|
|  4960.774193548387| 01|       193.84.66.147|
| 14210.416666666666| 01|dialup12.leuven.e...|
|  3448.818181818182| 01|dd15-053.compuser...|
|  4416.666666666667| 01|miriworld.its.uni...|
| 3246.8333333333335| 01|ix-esc-ca2-07.ix....|
|             4287.0| 01|      133.102.242.50|
|             3640.0| 01| tyo1.gate.nec.co.jp|
|  8159.142857142857| 01|ix-dgr-il1-15.ix....|
| 15684.777777777777| 01|      198.248.59.123|
| 6405.3421052631575| 01|hsccs_gatorbox07....|
|             2533.0| 01|mars.sunmtm.kuleu...|
|  7508.952380952381| 01|piweba4y.prodigy.com|
| 3357.1111111111113| 01|   unix.adeptcom.com|
|           23000.25| 01|port42.ts2.msstat...|
|  6676.71428

__3.9 Mostrar una lista de 40 endpoints distintos que generan código de respuesta = 404__

In [27]:
Ejercicio9 = sqlContext.sql("""
    SELECT DISTINCT (Endpoint), code as Codigo
    FROM datos
    WHERE code = 404
    LIMIT 40 
""")
Ejercicio9.show()

+--------------------+------+
|            Endpoint|Codigo|
+--------------------+------+
|/history/apollo/a...|   404|
|/history/apollo/a...|   404|
|/shuttle/resource...|   404|
|/history/apollo/a...|   404|
|/elv/DELTA/uncons...|   404|
|/pub/winvn/readme...|   404|
|/history/apollo/a...|   404|
|/history/history.htm|   404|
|     /sts-71/launch/|   404|
|/history/apollo/a...|   404|
|/www/software/win...|   404|
|/history/apollo/a...|   404|
|/pub/winvn/releas...|   404|
+--------------------+------+



__3.10 Mostrar el top 25 de endpoints que más códigos de respuesta 404 generan__

In [51]:
Ejercicio10 = sqlContext.sql ("""
    SELECT Endpoint COUNT (Code) AS Codigo 
    FROM datos
    WHERE Code = 404
    GROUP BY Endpoint 
    ORDER BY (Code) DESC
    LIMIT = 25
""")

Ejercicio10.show()


ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ParseException: "\nmismatched input '(' expecting {<EOF>, ',', 'FROM', 'WHERE', 'GROUP', 'ORDER', 'HAVING', 'LIMIT', 'LATERAL', 'WINDOW', 'UNION', 'EXCEPT', 'INTERSECT', 'SORT', 'CLUSTER', 'DISTRIBUTE'}(line 2, pos 26)\n\n== SQL ==\n\n    SELECT Endpoint COUNT (Code) AS Codigo \n--------------------------^^^\n    FROM datos\n    WHERE Code = 404\n    GROUP BY Endpoint \n    ORDER BY (Code) DESC\n    LIMIT = 25\n"

__3.11 El top 5 de días que se generaron código de respuestas 404__

In [56]:
Ejercicio11 = sqlContext.sql("""
    SELECT (Code) as codigo COUNT (date) AS Dia 
    FROM datos
    WHERE Code = 404
    ORDER BY (date) DESC
    LIMIT 5
""")
Ejercicio11.show()


ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ParseException: "\nmismatched input 'COUNT' expecting {<EOF>, ',', 'FROM', 'WHERE', 'GROUP', 'ORDER', 'HAVING', 'LIMIT', 'LATERAL', 'WINDOW', 'UNION', 'EXCEPT', 'INTERSECT', 'SORT', 'CLUSTER', 'DISTRIBUTE'}(line 2, pos 28)\n\n== SQL ==\n\n    SELECT (Code) as codigo COUNT (date) AS Dia \n----------------------------^^^\n    FROM datos\n    WHERE Code = 404\n    ORDER BY (date) DESC\n    LIMIT 5\n"